# Exemplo 1 - Redução de dimensão

Neste exemplo iremos analisar dados agregados do último censo.

Será que conseguiremos resumir os dados apresentados em poucas visualizações?

## Carregar pacotes

In [ ]:
library(tidyverse)
library(magrittr)
library(plotly)
library(psych)

## Carregar dados

In [ ]:
dados_municipios <- read_csv("/home/vm-data-science/dados/dados_municipios.csv")

In [ ]:
dados_municipios %>% head

## Análises

### *Principal Components Analysis* - PCA

Agora que podemos criar 2 índices e observar o conceito de rotação.

In [ ]:
pca_model_norotate <- principal(dados_municipios %>% 
                        select( contains('t_') ), 
                       nfactors = 8, 
                       rotate = "none" )

In [ ]:
pca_model_norotate

In [ ]:
pca_model_rotate <- principal(dados_municipios %>% 
                        select( contains('t_') ), 
                       nfactors = 8, 
                       rotate = "varimax" )

In [ ]:
pca_model_rotate

In [ ]:
dados_municipios %<>% 
  mutate( PC1 = pca_model_norotate$scores[,1],
          PC2 = pca_model_norotate$scores[,2],
          PC1_rot = pca_model_rotate$scores[,1],
          PC2_rot = pca_model_rotate$scores[,2] )

In [ ]:
dados_estados <- dados_municipios %>% 
  group_by( estado ) %>% 
  summarise( PC1 = mean(PC1), 
             PC2 = mean(PC2),
             PC1_rot = mean(PC1_rot), 
             PC2_rot = mean(PC2_rot) )

- visão não rotacionada

In [ ]:
p <- plot_ly() %>%
    add_trace(x=~dados_estados$PC1, 
              y=~dados_estados$PC2,
              type="scatter",
              mode="markers" ) 
p

In [ ]:
variaveis <- pca_model_norotate$loadings[] %>% rownames()

for( k in 1:nrow(pca_model_norotate$loadings) ) {
    
    x <- c(0, pca_model_norotate$loadings[k,1])
    y <- c(0, pca_model_norotate$loadings[k,2])
    p <- p %>% add_trace(x=x, y=y,
                     type="scatter", 
                     mode="lines",
                     line = list(width=1),
                     opacity = 1,
                     name = variaveis[k] )
}

In [ ]:
p

- visão rotacionada

In [ ]:
p2 <- plot_ly() %>%
    add_trace(x=~dados_estados$PC1_rot, 
              y=~dados_estados$PC2_rot,
              type="scatter",
              mode="markers" ) 

In [ ]:
variaveis <- pca_model_rotate$loadings[] %>% rownames()

for( k in 1:nrow(pca_model_rotate$loadings) ) {
    
    x <- c(0, pca_model_rotate$loadings[k,1])
    y <- c(0, pca_model_rotate$loadings[k,2])
    p2 <- p2 %>% add_trace(x=x, y=y,
                     type="scatter", 
                     mode="lines",
                     line = list(width=1),
                     opacity = 1,
                     name = variaveis[k]  )
}

In [ ]:
p2